---
# Check if GPU is available

In [1]:
import torch
torch.cuda.is_available()

False

---
# Logger

In [2]:
from lightning.pytorch.loggers import NeptuneLogger
from dotenv import load_dotenv
import os

API_TOKEN = os.environ.get("LOGGER_API")
PROJECT_NAME = 'mrobay/Audio-project'

logger_config = {
    "api_key": API_TOKEN,
    "project_name": PROJECT_NAME,
    "log_model_checkpoints": False
}

---
# Loading Data

In [3]:
import importlib
imported_module = importlib.import_module("dataModule")
importlib.reload(imported_module)
from dataModule import AudioDataModule
dm = AudioDataModule()
dm.prepare_data()
dm.setup("fit")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning_sdk/helpers.py:44: A newer version of lightning-sdk is available (0.1.15). Please consider upgrading with `pip install -U lightning-sdk`. Not all functionalities of the platform can be guaranteed to work with the current version.


# Create the classifier and train it

In [ ]:
import importlib
importlib.reload(importlib.import_module("hebrewASR"))
importlib.reload(importlib.import_module("dataModule"))
from hebrewASR import HebrewASR, train_func, default_config
neptune_logger = NeptuneLogger(project=PROJECT_NAME, api_key=API_TOKEN, log_model_checkpoints=False)
trainer = train_func(config=default_config, logger=neptune_logger)

# Validation

---
# Test the classifier

### prepre the test data

In [16]:
import importlib
imported_module = importlib.import_module("dataModule")
importlib.reload(imported_module)
from dataModule import AudioDataModule
dm_test = AudioDataModule()
dm_test.prepare_data()
dm_test.setup("test")

### Trainer

In [23]:
from lightning.pytorch.loggers import NeptuneLogger
from dotenv import load_dotenv
import os

API_TOKEN = os.environ.get("LOGGER_API")
PROJECT_NAME = 'mrobay/Audio-project'

logger_config = {
    "api_key": API_TOKEN,
    "project_name": PROJECT_NAME,
    "log_model_checkpoints": False
}

neptune_logger_test = NeptuneLogger(project=PROJECT_NAME, api_key=API_TOKEN, log_model_checkpoints=False)

In [24]:
import lightning.pytorch as pl
trainer_test = pl.Trainer(
        devices="auto",
        accelerator="auto",
        logger=neptune_logger_test)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


### Load the model

In [25]:
import importlib
importlib.reload(importlib.import_module("hebrewASR"))
importlib.reload(importlib.import_module("dataModule"))
from hebrewASR import HebrewASR, train_func, default_config

model_test = HebrewASR.load_from_checkpoint("/teamspace/studios/this_studio/.neptune/AUD1-223/AUD1-223/checkpoints/epoch=177-step=356.ckpt")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/home/zeus/miniconda3/envs/cloudsp

### Test the model

In [26]:
trainer_test.test(model_test, datamodule=dm_test)
neptune_logger_test.run.stop()

[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mrobay/Audio-project/e/AUD1-228


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:122: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.


Testing: |          | 0/? [00:00<?, ?it/s]

['', 'י היוהזי יה ההה הזי  ה אגיוהה י', 'ההי י הההה זיהה הי', '', 'ה הה זה הזיהזיהי י תהיזבליה זיה הזהי  ה הוהיהי יוה זיה בזהאיהזיה זליההביי בי זיייזיתהזידההיה הזה הה זה זיזהיויה ייהזיההזהה הה ההולה ה ביהזליי', '', 'ההזלהי היההההה הזיוהיה ו זי זיבליההזההיה הלי ה ה הזיזהיהזה יהיהזיתההודי הזגיה באיתההזיתההה זיה זי וליההיה הזהלהזיההה', '', '', ' יהתדוהזייבלהיהזידוההה הזיבליהי י תההביזהווהזיה בזיהיזההי ה זיולהיהתהיה היהיבלי', 'היה היהי פיה זיוהזייהיזהיהז', 'יההזהה ביהיולההבלייה זי מאיזה לגדי הזיה זי ה מאידוהולה זה לה הזה', ' היה הזיי הזיהולהזה  הי י בל', '', 'הזיובהזי הזהייו ה יהיושהזיההתוהזהליה ה הביהוה איהתהיו', '', '', 'ההה שעלהיויהתהזניוהיזלהיאיהתהזהתהניזהיהיתזהושלשהיזההגנייושאתאתובליאיאתההז להתהייושהתדלהליושהזיבדליהאיושאתהזהוהתהזליויביההה', 'הה זיהיוליוה יהההזהה הזה הלהיה ייותההי ביה   הזילההיהיי', ' זיזדליאביזההזיזיהיה הולזיזההזי הזי זלזה י ההאבלהיתיזהוב ה זיה תה  בזהה יי איתהיזידיה ביזהולה אילהי היה זיבי ה זייתולהזהליהלהי', '', 'יהיהזהי דלהוגיי  גיה  זיהה  ', ' אידהיהגדוהה יי הזיה',